In [1]:
print("ok")

ok


In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [4]:
extracted_data=load_pdf_file(data="C:\\Users\\vinay\\OneDrive\\Desktop\\New folder (3)\\Data")

In [5]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [6]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

In [9]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings
embeddings = download_hugging_face_embeddings()

C:\Users\vinay\AppData\Local\Temp\ipykernel_1908\3883074817.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\vinay\miniconda3\envs\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [19]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_3xBdCP_4pWeFB6PgW2tyUg9Amoxc4RwCYLa9h3cPiiyaXcsJzeCTceUuhg5Z6aJGjQkZ7M")

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [19]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_3xBdCP_4pWeFB6PgW2tyUg9Amoxc4RwCYLa9h3cPiiyaXcsJzeCTceUuhg5Z6aJGjQkZ7M"
index_name = "medicalbot"


In [14]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [15]:
docsearch

In [3]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

c:\Users\vinay\miniconda3\envs\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


NameError: name 'index_name' is not defined

In [17]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [41]:
retrieved_docs = retriever.invoke("what is acne?")

In [34]:
retrieved_docs

[Document(id='4d0f8002-bd36-4452-8757-37d7c0b184ec', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 234.0, 'page_label': '235', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Users\\vinay\\OneDrive\\Desktop\\New folder (3)\\Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Definition\nAntepartum testing consists of a variety of tests per-\nformed late in pregnancy to verify fetal well-being, as\njudged by the baby’s heart rate and other characteristics.\nAntepartum tests include the nonstress test (NST), bio-\nphysical profile, and contraction stress test (CST).\nPurpose\nAntepartum testing is performed after 32 weeks of\npregnancy so that the couple and the doctor can be\nKEY TERMS\nUltrasound— A device that records sound waves'),
 Document(id='1f62944b-058a-4681-890b-df102d5f529f', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06

In [ ]:
import requests

GROQ_API_KEY = "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"
GROQ_ENDPOINT = "https://api.groq.com/openai/v1/chat/completions"



def generate_response(query, retrieved_docs):
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    payload = {
        "model": "llama3-8b-8192",
        "messages": [
            {"role": "system", "content": "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise."},
            {"role": "user", "content": f"Query: {query}\n\nContext: {context}"}
        ]
    }

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    response = requests.post(GROQ_ENDPOINT, headers=headers, json=payload)
    
    return response.json().get("choices", [{}])[0].get("message", {}).get("content", "Error: Unexpected response from API.")

# Retrieve documents only once
question = "what is acne?"
retrieved_docs = retriever.invoke(question)

answer_1 = generate_response(question, retrieved_docs)


print("\nAI Response 1:\n", answer_1)




AI Response 1:
 Acne is a skin disorder characterized by the inflammation of sebaceous glands. It is the general name given to a condition where the oil-producing glands in the skin, called sebaceous glands, become inflamed. This inflammation leads to the formation of pimples, blackheads, and other blemishes on the skin.
